<a href="https://colab.research.google.com/github/jnises/llmog/blob/finetuning/quantize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
model_name = 'gemma-3-1b-llmog'
quantized_model_name = 'gemma-3-1b-llmog-GGUF'
HF_REPO = f"jnises/{model_name}"
QUANTIZED_HF_REPO = f"jnises/{quantized_model_name}"

In [ ]:
from unsloth import FastModel
#import torch

# fourbit_models = [
#     # 4bit dynamic quants for superior accuracy and low memory use
#     "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
#     "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
#     "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
#     "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

#     # Other popular models!
#     "unsloth/Llama-3.1-8B",
#     "unsloth/Llama-3.2-3B",
#     "unsloth/Llama-3.3-70B",
#     "unsloth/mistral-7b-instruct-v0.3",
#     "unsloth/Phi-4",
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    #model_name = "unsloth/gemma-3-4b-it",
    model_name = HF_REPO,
    #max_seq_length = 2048, # Choose any for long context!
    #max_seq_length = 4096, # try with larger sequences
    #load_in_4bit = True,  # 4 bit quantization to reduce memory
    #load_in_4bit = False, # not needed on l4?
    #load_in_8bit = True, # [NEW!] A bit more accurate, uses 2x memory
    #full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models

)


In [ ]:
model.save_pretrained_merged(quantized_model_name, tokenizer)

In [ ]:
model.save_pretrained_gguf(
        # need to be just the model name here, a full path results in confusing error messages
        quantized_model_name,
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
        #quantization_type='q4_k_m',
    )

In [ ]:
# model.push_to_hub_gguf(
#         quantized_model_name,
#         quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
#         repo_id = QUANTIZED_HF_REPO,
#         token = HF_TOKEN,
#     )

In [ ]:
%ls

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
api.upload_file(path_or_fileobj='gemma-3-1b-llmog-GGUF.Q8_0.gguf', path_in_repo='gemma-3-1b-llmog-GGUF.Q8_0.gguf', repo_id=QUANTIZED_HF_REPO, repo_type='model', token=HF_TOKEN
)